# コード7-2：個体ベースシミュレーションの数値計算の実装

『ウイルス感染の数理モデルとシミュレーション ーデータを定量的に理解するー』

個体ベースシミュレーションの数値計算の実装の例をまとめる。

## ライブラリ、パッケージ（モジュール）の読み込み

In [1]:
## 必要があればインストール
# install.packages(c("ggplot2", "patchwork", "tidyr"))

In [158]:
library(ggplot2)
library(patchwork)
library(tidyr)



- M Mahgoub et al. Sporadic on/off switching of HTLV-1 Tax expression is crucial to maintain the whole population of virus-induced leukemic cells. PNAS 2018  
https://doi.org/10.1073/pnas.1715724115

- 論文中の個体ベースシミュレーションの数値計算のコード  
https://github.com/petadimension/Tax_dynamics/


初期状態

day 3

104 MT-1細胞（shRNAあり）  
63 MT-1細胞（shRNAなし）

shNC  
Tax<sub>on</sub>が10%より小さい  
72%がTax<sub>off</sub> A<sub>high</sub>

shTax4 Tax<sub>on</sub>が10%より小さい

実行時間

day 3 - day 21

<img src="../../assets/Mahgoub_etal_2018_Fig5.jpeg" alt="" width="400px"> <img src="../../assets/Mahgoub_etal_2018_FigS6.png" alt="" width="400px">   


### 細胞群

通常のMT-1細胞  
- Tax<sub>on</sub>の細胞：$T_{\text{on}}$ 
- Tax<sub>off</sub>かつ抗アポトーシスが高い状態の細胞：$T_{\text{off}} A_{\text{high}}$  
- Tax<sub>off</sub>かつ抗アポトーシスが低い状態の細胞：$T_{\text{off}} A_{\text{low}}$ 

shRNAを導入したMT-1細胞  
- Tax<sub>on</sub>の細胞：$T^{*}_{\text{on}}$ 
- Tax<sub>off</sub>かつ抗アポトーシスが高い状態の細胞：$T^{*}_{\text{off}} A_{\text{high}}$  
- Tax<sub>off</sub>かつ抗アポトーシスが低い状態の細胞：$T^{*}_{\text{off}} A_{\text{low}}$ 

### 細胞群の出生・死亡

通常のMT-1細胞 

- Tax<sub>on</sub>の細胞の増殖：$g_{\text{on}} T_{\text{on}}$  
- Tax<sub>off</sub>かつ抗アポトーシスが高い状態の細胞の増殖：$g T_{\text{off}} A_{\text{high}}$  
- Tax<sub>off</sub>かつ抗アポトーシスが低い状態の細胞の増殖：$g T_{\text{off}} A_{\text{low}}$  
- Tax<sub>off</sub>の細胞の抗アポトーシスが高い状態から低い状態への遷移：$p T_{\text{off}} A_{\text{high}}$  
- Tax<sub>off</sub>かつ抗アポトーシスが低い状態の細胞の死亡：$\delta T_{\text{off}} A_{\text{low}}$  


shRNAを導入したMT-1細胞  

- Tax<sub>on</sub>の細胞の増殖：$g_{\text{on}} T^{*}_{\text{on}}$  
- Tax<sub>off</sub>かつ抗アポトーシスが高い状態の細胞の増殖：$g T^{*}_{\text{off}} A_{\text{high}}$  
- Tax<sub>off</sub>かつ抗アポトーシスが低い状態の細胞の増殖：$g T^{*}_{\text{off}} A_{\text{low}}$  
- Tax<sub>off</sub>の細胞の抗アポトーシスが高い状態から低い状態への遷移：$p^{*} T^{*}_{\text{off}} A_{\text{high}}$  
- Tax<sub>off</sub>かつ抗アポトーシスが低い状態の細胞の死亡：$\delta T^{*}_{\text{off}} A_{\text{low}}$ 

### Taxの発現（＝時間分布）

- Taxを発現する期間：$t_{\text{period}}$
- Taxを発現しない期間：$t_{\text{interval}}$

### Tax<sub>on</sub>の細胞のTax発現が終わるまでの時間

- Tax<sub>on</sub>の細胞が分裂してできた新規娘細胞：$t_{\text{period}}$からランダムサンプリング
- Tax<sub>off</sub>からTax<sub>on</sub>になった細胞：$t_{\text{period}}$からランダムサンプリング
- そのほかのTax<sub>on</sub>の細胞：$t_{\text{period}}$からTax発現後の時間を引いた時間

In [71]:
## 状態の取得

get_alivecell_IDs <- function( lpop ) {
	ID_vec <- lpop[[1]]
	alive_IDs <- which( is.na(ID_vec) == FALSE )
	return( alive_IDs  )
}

get_deadcell_IDs <- function( lpop ) {
	ID_vec <- lpop[[1]]
	dead_IDs <- which( is.na(ID_vec) == TRUE )
	return( dead_IDs )
}

get_TaxOn_IDs <- function( lpop ) {
	state_vec <- lpop[[2]]
	OnIDs <- which( state_vec == "TaxOn" )
	aidx <- get_alivecell_IDs(lpop)
	OnIDs <- intersect(OnIDs, aidx)
	return( OnIDs )
}

get_TaxOn_sh_IDs <- function( lpop ) {
	state_vec <- lpop[[2]]
	OnIDs <- which( state_vec == "TaxOn_sh" )
	aidx <- get_alivecell_IDs(lpop)
	OnIDs <- intersect(OnIDs, aidx)
	return( OnIDs )
}

get_TaxOff_IDs <- function( lpop ) {
	state_vec <- lpop[[2]]
	OffIDs <- which( state_vec == "TaxOff" )
	aidx <- get_alivecell_IDs(lpop)
	OffIDs <- intersect(OffIDs, aidx)
	return( OffIDs )
}

get_TaxOff_sh_IDs <- function( lpop ) {
	state_vec <- lpop[[2]]
	OffIDs <- which( state_vec == "TaxOff_sh" )
	aidx <- get_alivecell_IDs(lpop)
	OffIDs <- intersect(OffIDs, aidx)
	return( OffIDs )
}

get_Apoptotic_IDs <- function( lpop ) {
	state_vec <- lpop[[2]]
	ApoIDs <- which( state_vec == "Apop" )
	aidx <- get_alivecell_IDs(lpop)
	ApoIDs <- intersect(ApoIDs, aidx)
	return( ApoIDs )
}

get_Apoptotic_sh_IDs <- function( lpop ) {
	state_vec <- lpop[[2]]
	ApoIDs <- which( state_vec == "Apop_sh" )
	aidx <- get_alivecell_IDs(lpop)
	ApoIDs <- intersect(ApoIDs, aidx)
	return( ApoIDs )
}

get_TaxOnhist_IDs <- function( lpop ) {
    state_vec <- lpop[[2]]
	hist_vec <- lpop[[4]]
    stateIDs <- which(state_vec %in% c("TaxOn", "TaxOff", "Apop"))
	histIDs <- which( hist_vec == "yes" )
	aidx <- get_alivecell_IDs(lpop)
	histIDs <- intersect(intersect(stateIDs, histIDs), aidx)
	return( histIDs )
}

get_TaxOnhist_sh_IDs <- function( lpop ) {
    state_vec <- lpop[[2]]
	hist_vec <- lpop[[4]]
    stateIDs <- which(state_vec %in% c("TaxOn_sh", "TaxOff_sh", "Apop_sh"))
	histIDs <- which( hist_vec == "yes" )
	aidx <- get_alivecell_IDs(lpop)
	histIDs <- intersect(intersect(stateIDs, histIDs), aidx)
	return( histIDs )
}

In [85]:
## 状態の更新

death_process <- function( lpop, tID ) {
	ID_vec <- lpop[[1]]
	ID_vec[tID] <- NA
	lpop[[1]] <- ID_vec
	return( lpop )
}

division_process <- function( lpop, mID, dID ) {
	ID_vec <- lpop[[1]]
	state_vec <- lpop[[2]]
	clock_vec <- lpop[[3]]
	hist_vec <- lpop[[4]]
	ID_vec[dID] <- dID
	state_vec[dID] <- state_vec[mID]
	hist_vec[dID] <- hist_vec[mID]
	if( state_vec[mID] %in% c("TaxOn", "TaxOn_sh") ) {
		clock_vec[dID] <- sample(t_periods, 1, replace = TRUE)
	} else if( state_vec[mID] %in% c("TaxOff", "Apop") ) {
		clock_vec[dID] <- sample(t_intervals, 1, replace = TRUE)
	} else {
        clock_vec[dID] <- sample(t_intervals_sh, 1, replace = TRUE)
    }
    
	lpop[[1]] <- ID_vec
	lpop[[2]] <- state_vec
	lpop[[3]] <- clock_vec
	lpop[[4]] <- hist_vec
	return( lpop )
}

newborn_registry <- function( lpop, newID ) {
	ID_vec <- lpop[[1]]
	state_vec <- lpop[[2]]
	clock_vec <- lpop[[3]]
	hist_vec <- lpop[[4]]
	ID_vec <- c(ID_vec, newID)
	state_vec <- c(state_vec, NA)
	clock_vec <- c(clock_vec ,NA)
	hist_vec <- c(hist_vec, NA)
	lpop[[1]] <- ID_vec
	lpop[[2]] <- state_vec
	lpop[[3]] <- clock_vec
	lpop[[4]] <- hist_vec
	return( lpop )
}

update_clock <- function( lpop, time_step ) {
	clock_vec <- lpop[[3]]
	aidx <- get_alivecell_IDs( lpop )
	clock_vec[aidx] <- clock_vec[aidx] - time_step
	lpop[[3]] <- clock_vec
	return( lpop )
}

change_TaxOnOff <- function( lpop ) {
	state_vec <- lpop[[2]]
	clock_vec <- lpop[[3]]
	hist_vec <- lpop[[4]]
	tidx <- which( clock_vec < 0 )
	if( length(tidx) > 0 ) {
		for( tid in tidx ) {
			if( state_vec[tid] == "TaxOn" ) {
				state_vec[tid] <- "TaxOff"
				clock_vec[tid] <- sample(t_intervals, 1, replace = TRUE)
			} else if( state_vec[tid] == "TaxOn_sh" ) {
				state_vec[tid] <- "TaxOff_sh"
				clock_vec[tid] <- sample(t_intervals_sh, 1, replace = TRUE)
			} else if( state_vec[tid] == "TaxOff" ) {
                state_vec[tid] <- "TaxOn"
                clock_vec[tid] <- sample(t_periods, 1, replace = TRUE)
                hist_vec[tid] <- "yes"
			} else if( state_vec[tid] == "TaxOff_sh" ) {
                state_vec[tid] <- "TaxOn_sh"
                clock_vec[tid] <- sample(t_periods, 1, replace = TRUE)
                hist_vec[tid] <- "yes"
			} else if( state_vec[tid] == "Apop" ) {
                state_vec[tid] <- "TaxOn"
                clock_vec[tid] <- sample(t_periods, 1, replace = TRUE)
                hist_vec[tid] <- "yes"
			} else if( state_vec[tid] == "Apop_sh" ) {
                state_vec[tid] <- "TaxOn_sh"
                clock_vec[tid] <- sample(t_periods, 1, replace = TRUE)
                hist_vec[tid] <- "yes"
			}
		}
	}
	lpop[[2]] <- state_vec
	lpop[[3]] <- clock_vec
	lpop[[4]] <- hist_vec
	return( lpop )
}

change_TaxOff2Apo <- function( lpop, tid ) {
	state_vec <- lpop[[2]]
	clock_vec <- lpop[[3]]
    if (state_vec[tid] == "TaxOff") {
        state_vec[tid] <- "Apop"
        clock_vec[tid] <- sample(t_intervals, 1, replace = TRUE)
    } else if (state_vec[tid] == "TaxOff_sh") {
        state_vec[tid] <- "Apop_sh"
        clock_vec[tid] <- sample(t_intervals_sh, 1, replace = TRUE)
    }
	
	lpop[[2]] <- state_vec
	lpop[[3]] <- clock_vec
	return( lpop )
}

In [171]:
## シミュレーションの設定

seedn <- 123 #123456
set.seed(seedn) # set a seed of random number generator
Tmin <- 3.0 # start of experiment
Tmax <- 21.0 # 21.0 # 4.0 # (hours) end of experiment
min_step <- 1.0

skipnum <- 10000
Nthr <- 10000

In [172]:
## 初期条件とパラメータ

Tstart <- 2.979515

NonInit <- 6
NonInit_sh <- 6

NoffInit <- 80
list_NoffInit_sh <- list(49, 1) # shNCは49、shTax4は1

NapoInit <- 18
list_NapoInit_sh <- list(8, 56) # shNCは8、shTax4は56

Ninit <- NonInit + NoffInit + NapoInit + NonInit_sh + NoffInit_sh + NapoInit_sh # total number of cells per well

gon <- 0.54 # 0.52 # cell division rate of TaxOn cells (1/day)
g <- 0.68 #0.65 # cell division rate (1/day)
delta <- 0.25 # 0.6 # additional death via apoptosis (1/day)
p <- 0.02 # 0.05 # transition from TaxOff to Apoptotic cell
list_p_sh <- list(0.02, 0.4) # shNCは0.02、shTax4は0.4

In [173]:
## Tax発現時間の発現間隔の分布の読み込み

t_periods <- read.table("https://raw.github.com/petadimension/Tax_dynamics/master/ABM/sim_tax_period_sampling/optim_t_period.txt", header = F)$V1
t_intervals <- read.table("https://raw.github.com/petadimension/Tax_dynamics/master/ABM/sim_tax_period_sampling/optim_t_interval.txt", header = F)$V1

t_periods <- t_periods/24.0 # convert to day unit
t_intervals <- t_intervals/24.0 # convert to day unit

## t_periodsの3時間以下および60時間以上の切り捨て
over60idx <- which( t_periods*24.0>60.0 )
if( length(over60idx)>0 ) {
    t_periods <- t_periods[-over60idx]
}
less3idx <- which( t_periods*24.0<3.0 )
if( length(less3idx)>0 ) {
    t_periods <- t_periods[-less3idx]
}


list_t_intervals_sh <- list(t_intervals, Inf) # shTax4ではInf

In [ ]:
time_start <- Sys.time()

for (i in 1:2) {
    
    print(paste(i, ":", Sys.time()))
    
    ## shRNA固有の条件の切り替え
    NoffInit_sh <- list_NoffInit_sh[[i]]
    NapoInit_sh <- list_NapoInit_sh[[i]]
    p_sh <- list_p_sh[[i]]
    t_intervals_sh <- list_t_intervals_sh[[i]]

    
    ## 集団の情報をリストに保存

    IDs <- 1:Ninit

    InitialOnIDs <- 1:NonInit
    InitialOffIDs <- (NonInit + 1):(NonInit + NoffInit)
    InitialApoIDs <- (NonInit + NoffInit + 1):(NonInit + NoffInit + NapoInit)

    InitialOnshIDs <- (NonInit + NoffInit + NapoInit + 1):(NonInit + NoffInit + NapoInit + NonInit_sh)
    InitialOffshIDs <- (NonInit + NoffInit + NapoInit + NonInit_sh + 1):(NonInit + NoffInit + NapoInit + NonInit_sh + NoffInit_sh)
    InitialAposhIDs <- (NonInit + NoffInit + NapoInit + NonInit_sh + NoffInit_sh + 1):(NonInit + NoffInit + NapoInit + NonInit_sh + NoffInit_sh + NapoInit_sh)


    states <- rep("", length = Ninit)
    states[InitialOnIDs] <- "TaxOn"
    states[InitialOffIDs] <- "TaxOff"
    states[InitialApoIDs] <- "Apop"
    states[InitialOnshIDs] <- "TaxOn_sh"
    states[InitialOffshIDs] <- "TaxOff_sh"
    states[InitialAposhIDs] <- "Apop_sh"

    clocks <- rep(0.0, length = Ninit)
    clocks[InitialOnIDs] <- sample(t_periods, length(InitialOnIDs), replace = TRUE)
    clocks[InitialOffIDs] <- sample(t_intervals, length(InitialOffIDs), replace = TRUE)
    clocks[InitialApoIDs] <- sample(t_intervals, length(InitialApoIDs), replace = TRUE)

    clocks[InitialOnshIDs] <- sample(t_periods, length(InitialOnshIDs), replace = TRUE)
    clocks[InitialOffshIDs] <- sample(t_intervals_sh, length(InitialOffshIDs), replace = TRUE)
    clocks[InitialAposhIDs] <- sample(t_intervals_sh, length(InitialAposhIDs), replace = TRUE)

    histories <- rep("", length = Ninit)
    histories[InitialOnIDs] <- "yes"
    histories[InitialOffIDs] <- "no"
    histories[InitialApoIDs] <- "no"

    histories[InitialOnshIDs] <- "yes"
    histories[InitialOffshIDs] <- "no"
    histories[InitialAposhIDs] <- "no"

    dfpop <- data.frame(ID = IDs, state = states, clock = clocks, TaxOnhistory = histories, stringsAsFactors = FALSE)
    lpop <- as.list(dfpop)


    ## 初期化
    t <- Tstart
    Nalive <- Ninit
    rates <- c(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    count <- 0

    ## 初期状態の出力 ##
    if( !file.exists("outputs/") ) dir.create("outputs/")
    sfn <- paste("outputs/TaxOnOffApo_num_", Ninit, "_seed_", seedn, "_max_t_interval_", max(t_intervals_sh), "_p_sh_", p_sh, ".txt", sep = "")
    pout <- "time\tNOn\tNOff\tApo\thist\tNOn_sh\tNOff_sh\tApo_sh\thist_sh\n"
    cat(pout, file = sfn, append = FALSE)

    pout <- sprintf("%f\t%d\t%d\t%d\t%d\t%d\t%d\t%d\t%d\n",
                    t, length(get_TaxOn_IDs(lpop)), length(get_TaxOff_IDs(lpop)), length(get_Apoptotic_IDs(lpop)), length(get_TaxOnhist_IDs(lpop)),
                    length(get_TaxOn_sh_IDs(lpop)), length(get_TaxOff_sh_IDs(lpop)), length(get_Apoptotic_sh_IDs(lpop)), length(get_TaxOnhist_sh_IDs(lpop)))
    cat(pout, file = sfn, append = TRUE)


    ## シミュレーション

    

    while ( t < Tmax ) {
        if ( Nalive > 0 ) {


            Ons <- get_TaxOn_IDs(lpop)
            Offs <- get_TaxOff_IDs(lpop)
            Apos <- get_Apoptotic_IDs(lpop)
            hists <- get_TaxOnhist_IDs(lpop)

            Ons_sh <- get_TaxOn_sh_IDs(lpop)
            Offs_sh <- get_TaxOff_sh_IDs(lpop)
            Apos_sh <- get_Apoptotic_sh_IDs(lpop)
            hists_sh <- get_TaxOnhist_sh_IDs(lpop)

            Non <- length(Ons)
            Noff <- length(Offs)
            Napo <- length(Apos)
            Nhist <- length(hists)

            Non_sh <- length(Ons_sh)
            Noff_sh <- length(Offs_sh)
            Napo_sh <- length(Apos_sh)
            Nhist_sh <- length(hists_sh)


            ## shRNAが導入されていないMT-1細胞の数が`Nthr`を超えた場合、イベントのレートを`skipnum`回ごとに行う
            if ( Nalive > Nthr ) {
                if ( count > skipnum ) {
                    rates[1] <- gon*(Non + Non_sh) #g*Non
                    rates[2] <- g*(Noff + Noff_sh)
                    rates[3] <- g*(Napo + Napo_sh)
                    rates[4] <- delta*(Napo + Napo_sh)
                    rates[5] <- p*Noff
                    rates[6] <- p_sh*Noff_sh
                    trates <- sum(rates)
                    pp <- rates/trates
                    count <- 0
                } else {
                    count <- count + 1
                }
            } else {
                rates[1] <- gon*(Non + Non_sh) #g*Non
                rates[2] <- g*(Noff + Noff_sh)
                rates[3] <- g*(Napo + Napo_sh)
                rates[4] <- delta*(Napo + Napo_sh)
                rates[5] <- p*Noff
                rates[6] <- p_sh*Noff_sh
                trates <- sum(rates)
                pp <- rates/trates
                count <- 0
            }

            event <- sample(c("gon", "goff", "gap", "dapo", "tOff2Apo", "tOff2Apo_sh"), 1, prob = pp)

            if ( event == "gon" ) {
                ## division process of TaxOn cells ##
                motherID <- sample(c(Ons, Ons_sh), 1, replace = F) # choice of a mother cell
                daughterID <- Nalive + 1
                lpop <- newborn_registry(lpop, daughterID)
                lpop <- division_process(lpop, motherID, daughterID)

            } else if ( event == "goff" ) {
                ## division process of TaxOff cells ##
                motherID <- sample(c(Offs, Offs_sh), 1, replace = F) # choice of a mother cell
                daughterID <- Nalive + 1
                lpop <- newborn_registry(lpop, daughterID)
                lpop <- division_process(lpop, motherID, daughterID)

            } else if ( event == "gap" ) {
                ## division process of Apoptotitc cells ##
                motherID <- sample(c(Apos, Apos_sh), 1, replace = F) # choice of a mother cell
                daughterID <- Nalive + 1
                lpop <- newborn_registry(lpop, daughterID)
                lpop <- division_process(lpop, motherID, daughterID)

            } else if ( event == "dapo") {
                ## death process of apoptotic cells ##
                go2deathnote <- sample(c(Apos, Apos_sh), 1, replace = F)
                lpop <- death_process(lpop, go2deathnote)

            } else if( event == "tOff2Apo" ) {
                ## Transition from TaxOff to Apoptotic cells ##
                tID <- sample(Offs, 1, replace = F) # choice of a target cell
                lpop <- change_TaxOff2Apo(lpop, tID)
            } else if( event == "tOff2Apo_sh" ) {
                ## Transition from TaxOff to Apoptotic cells ##
                tID <- sample(Offs_sh, 1, replace = F) # choice of a target cell
                lpop <- change_TaxOff2Apo(lpop, tID)
            }

            ## calculate time step due to death ##
            dt <- rexp(1, rate = trates)
            time_step <- dt

            ## update status ##
            lpop <- change_TaxOnOff(lpop)
            lpop <- update_clock(lpop, time_step)

            aliveIDs <- get_alivecell_IDs(lpop)
            Nalive <- length(aliveIDs)

        } else {
            print( "No alive cells. Finish computation." )
            t <- Tmax
            next
        }

        t <- t + time_step
        out <- c(t, Non, Noff, Napo, Nhist, Non_sh, Noff_sh, Napo_sh, Nhist_sh)

        pout <- sprintf("%f\t%d\t%d\t%d\t%d\t%d\t%d\t%d\t%d\n",
                        out[1], out[2], out[3], out[4], out[5],
                        out[6], out[7], out[8], out[9])
        cat(pout, file = sfn, append = TRUE)
    }

}


time_end <- Sys.time()
print(time_end - time_start)

In [ ]:
data_plot <- data.frame()

for (i in 1:2) {
    
    ## shRNA固有の条件の切り替え
    NoffInit_sh <- list_NoffInit_sh[[i]]
    NapoInit_sh <- list_NapoInit_sh[[i]]
    p_sh <- list_p_sh[[i]]
    t_intervals_sh <- list_t_intervals_sh[[i]]
    
    
    data_plot <- rbind(data_plot,
                       cbind(read.csv(paste("outputs/TaxOnOffApo_num_", Ninit, "_seed_", seedn, "_max_t_interval_", max(t_intervals_sh), "_p_sh_", p_sh, ".txt", sep = ""), sep = "\t"),
                             type = c("shNC", "shTax4")[i]))
}



In [ ]:
plt1 <- ggplot(data = pivot_longer(subset(data_plot, type == "shNC"),
                                   cols = c(-time, -type, -hist, -hist_sh),
                                   names_to = "cells",
                                   values_to = "values"),
               aes(x = time, y = log10(values), color = factor(cells, levels = c("NOn", "NOff", "Apo", "NOn_sh", "NOff_sh", "Apo_sh")))) +
    geom_vline(xintercept = 3, linetype = "dashed") +
    geom_line() +
    scale_color_discrete(name = "Cell type") +
    scale_x_continuous(limits = c(0, 20)) +
    scale_y_continuous(limits = c(0, 7)) +
    theme_classic()

plt3 <- ggplot(data = data_plot, aes(x = time, y = 100*(NOn_sh + NOff_sh + Apo_sh)/(NOn + NOff + Apo + NOn_sh + NOff_sh + Apo_sh), color = type)) +
    geom_vline(xintercept = 3, linetype = "dashed") +
    geom_line() +
    scale_x_continuous(limits = c(0, 20)) +
    scale_y_continuous(limits = c(0, 50)) +
    theme_classic()

options(repr.plot.width = 15, repr.plot.height = 10, repr.plot.res = 50) ## プロットの描画サイズの設定
plt1 + plt3